# Integration Tests
This notebook demonstrates running agents via the registry with a stub engine for testing.

In [34]:
import os, sys, types
from pathlib import Path
from app.utils.agent_factory import build_agent
from app.utils.symbol_graph.symbol_service import SymbolService

# Fake engine to avoid OpenAI dependency
class FakeEngine:
    def __init__(self):
        self.calls = []
    def call_engine(self, prompt: str, config: dict):
        self.calls.append(prompt)
        return '# transformed code'

fake_mod = types.SimpleNamespace(OpenAiGpt4oAgentEngineRunner=FakeEngine)
sys.modules['app.utils.engine.openai_gpt4o_runner'] = fake_mod
os.environ['OPENAI_API_KEY'] = 'fake'

symbol_service = SymbolService(Path('.'))
log_path = Path('integration_logs/prompt_log.jsonl')
log_path.parent.mkdir(exist_ok=True)

## Test 1: Generator Agent
Run the generator agent on a sample file.

In [35]:
agent = build_agent('generator', symbol_service)
result1 = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': '1', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('Generator output:', result1['response'][:60])

Generator output: # transformed code


## Test: discriminator agent

In [36]:
agent = build_agent('discriminator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'discriminator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('discriminator result:', res['response'][:40])

discriminator result: # transformed code


## Test: mediator agent

In [37]:
agent = build_agent('mediator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'mediator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('mediator result:', res['response'][:40])

mediator result: # transformed code


## Test: patchor agent

In [38]:
agent = build_agent('patchor', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'patchor', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('patchor result:', res['response'][:40])

patchor result: # transformed code

# === AI-FIRST METAD


## Test: recommender agent

In [39]:
agent = build_agent('recommender', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('recommender result:', res['response'][:40])

recommender result: # transformed code


## Test: generator agent

In [40]:
agent = build_agent('generator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'generator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('generator result:', res['response'][:40])

generator result: # transformed code


## Test: discriminator agent

In [41]:
agent = build_agent('discriminator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'discriminator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('discriminator result:', res['response'][:40])

discriminator result: # transformed code


## Test: mediator agent

In [42]:
agent = build_agent('mediator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'mediator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('mediator result:', res['response'][:40])

mediator result: # transformed code


## Test: patchor agent

In [43]:
agent = build_agent('patchor', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'patchor', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('patchor result:', res['response'][:40])

patchor result: # transformed code

# === AI-FIRST METAD


## Test: recommender agent

In [44]:
agent = build_agent('recommender', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('recommender result:', res['response'][:40])

recommender result: # transformed code


## Log Verification

In [45]:
import json

lines = [json.loads(l) for l in log_path.read_text().splitlines()]
print('entries:', len(lines))

# Only include entries with 'agent_role'
agent_roles = {e['agent_role'] for e in lines if 'agent_role' in e}
print(agent_roles)


entries: 12
{'PatchAgent', 'RecommenderAgent'}


## Basic Analysis

In [46]:
from collections import Counter
counts=Counter(e['agent_role'] for e in lines)
print(counts)

Counter({'RecommenderAgent': 6, 'PatchAgent': 6})


In [47]:
for e in lines[:2]:
    if 'agent_role' in e and 'response' in e:
        print(e['agent_role'], '->', e['response'][:30])
    else:
        print("Skipped entry (missing keys):", e)


Skipped entry (missing keys): {'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'agent_role': 'RecommenderAgent', 'target': 'my_symbol', 'content': '# transformed code', 'originating_agent': 'RecommenderAgent', 'timestamp': '2025-05-17T10:18:03.636372Z'}
Skipped entry (missing keys): {'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'agent_role': 'RecommenderAgent', 'target': 'my_symbol', 'content': '# transformed code', 'originating_agent': 'RecommenderAgent', 'timestamp': '2025-05-17T10:18:04.236901Z'}
